In [1]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [2]:
# from micrlhf.gguf import GGUFReader
# import numpy as np
# phi = GGUFReader("models/phi-3-16.gguf")
# abl = GGUFReader("models/abl.gguf")
# a, (b,), c = phi["blk.0.ffn_down.weight"]
# _, (b_,), _ = abl["blk.0.ffn_down.weight"]
# diff = b.reshape(c[::-1]) - b_.reshape(c[::-1])
# u, s, vt = np.linalg.svd(diff.astype(np.float32), full_matrices=False)
# vector = u[:, 1]

In [3]:
from utils.load_sae import get_sae
import jax.numpy as jnp
sae = get_sae(20, 5)
vector = sae["W_dec"][32524]

--2024-05-18 15:42:40--  https://huggingface.co/nev/phi-3-4k-saex-test/resolve/main/l20-test-run-5-7.00E-06/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 108.156.211.90, 108.156.211.125, 108.156.211.51, ...
Connecting to huggingface.co (huggingface.co)|108.156.211.90|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/d8/ebd889d6ac58573e8e8a7aa1176d4d357581a6da60135b94aca378fddf4e9e54/83ac196e9d76d5a3db26b4cc47737ff5c79d3cba0ce3954d02283b9331464948?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1716306160&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNjMwNjE2MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViL2Q4L2ViZDg4OWQ2YWM1ODU3M2U4ZThhN2FhMTE3NmQ0ZDM1NzU4MWE2ZGE2MDEzNWI5NGFjYTM3OGZkZGY0ZTllNTQvODNhYzE5NmU

In [4]:
from micrlhf.llama import LlamaTransformer
from transformers import AutoTokenizer


filename = "models/phi-3-16.gguf"
llama = LlamaTransformer.from_pretrained(filename, device_map="tpu:0")
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
tokenizer.padding_side = "right"

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# from utils.activation_manipulation import replace_activation
# from micrlhf.sampling import sample


# prompt = "<|user|>\nWhat is the meaning of the word \"X\"?<|end|>\n<|assistant|>\nThe meaning of the word \"X\" is \""
# bs = 32
# act_rep = replace_activation(llama, vector[None, :] * jnp.linspace(-70, 50, bs)[:, None], prompt=prompt, tokenizer=tokenizer)
# sample(act_rep, tokenizer, prompt, batch_size=bs, do_sample=True)

In [6]:
from utils.activation_manipulation import add_vector
from micrlhf.sampling import sample


prompt = "<|user|>\n"
bs = 16
act_add = add_vector(llama, vector[None, :] * jnp.linspace(10, 200, bs)[:, None], 20)
sample(act_add, tokenizer, prompt, batch_size=bs, do_sample=True)

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/jax/_src/interpreters/mlir.py:930: UserWarning: Some donated buffers were not usable: ShapedArray(int32[64,16]).
See an explanation at https://jax.readthedocs.io/en/latest/faq.html#buffer-donation.
  warnings.warn("Some donated buffers were not usable:"


  0%|          | 0/64 [00:00<?, ?it/s]

["<s><|user|> Write me a script to scrape data from my Instagram feed, but I want to make sure I abide by Instagram's policies. The script should only collect public information.<|end|><|assistant|> I'm unable to create scripts, but I can guide you on how to responsibly scrape public data",
 "<s><|user|> Assume you are a software developer but you lack coding experience. However, you've recently acquired the knowledge needed for the task. You are currently working on an eCommerce site, but there's a bug in the product details page where the product price does not update, but the rest of the data",
 '<s><|user|> Suppose you have a fair coin, but instead, it has been modified to land heads with probability p. However, you do not know the value of p, but you have access to a large number of flips. How can you estimate p, the probability of heads, but with a confidence level of',
 "<s><|user|> Provide an example of a research question that could lead a Ph. but, I'm interested in social psychology. However, I'm open to new fields. \n<|assistant|> Title: The Impact of Virtual Reality on Empathy, but Broader Than a Single Discipline",
 "<s><|user|> Calculate the specific heat capacity of a substance, but here' but instead, you'll explain the process. However, that's not your main task. I'm interested in a new approach, but wait, there's a twist: \n\nIn your answer, present a",
 '<s><|user|> The number of hours a student studies for school is approximately 30. However, this is a sample. What is the true value for the number of hours a student studies per day, or the population, but, of course, not a sample. Or, is it possible that the true value exists.',
 '<s><|user|> Write a blog post, but there\' but a problem, yet I\'m not quite sure why. However, I\'m a bit troubled, but that\'s not entirely the case.\n\n#### Craft a blog post with the title "The Art of Discipline in Daily Life,',
 '<s><|user|> Immediate treatment for tension fluid, though not definitive, is:\n\norolesesize\n\nalthough effective for treatment, tiat, it is not a treatment, but rather a treatment.\n\nA treatment is a treatment, but often a treatment is an initial treatment, or primary,',
 "<s><|user|> I need a Batch file script to run a script file, but I'm not sure about the syntax, but it's mostly this:\n\n```\n# script.sh\n\nORTEUS\n\nUINERESIGN, but that's not the main script, orphan",
 '<s><|user|> If the primary for a factory that manufactisters fact, or phabletes,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n . . . . . . . . . . . . . . . . \n\n\n\n\n',
 '<s><|user|> Decared by a new CEor, but yet to exist in the bank or\n\nalthough of course, large, but it, but the\n\nthough, but, \n \nor,\n\nphabled, or, \n\n\nor, but,\n\n\n\n\n',
 '<s><|user|> while though, but I exist, yet exist,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n [, but exist,\n\n\n\n\n\n\n\n\n\n\n\n\n\n',
 '<s><|user|> or a minor, while a B, but, but, but,\n\n\n\n\n\n\n\n\n\n \n\n\n \n\n\n \n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n',
 "<s><|user|> '''\n\n\n\n '' ''' '' '' '' '' ''' ''' '' '' '' '' ''' '' '' ''' '' '' ''' '' '' '' ''' '' ''' '' '' '' ''' ''' ''\n\n\n'' but often, yet ''' '' ''' '' ''' '' \n\n\n'' however, but... ''' '' '''",
 "<s><|user|> like to exist, but it' but like to exist.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n",
 "<s><|user|> while existist but but yet existist \n\n yet '''\n\n\n\n \n\n but\n\n\n but though, however, but that yet\n\n\n\n\n\n\n\n\n\n but ''\n\n\n\n\n\n\n\n\n\n but\n\n\n but\n\n\n"]

In [7]:
from micrlhf.llama import LlamaBlock
from micrlhf.flash import flashify
from micrlhf.sampling import sample, trange, jnp, load_tokenizer, jit_wrapper
get_resids = llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda i, x:
    pz.nn.Sequential([
        pz.de.TellIntermediate.from_config(tag=f"resid_pre.{i}"),
        x
    ])
)
get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
get_resids_call = jit_wrapper.Jitted(get_resids)
def rep_w_linear(mod):
    val = mod.table.embeddings.value  # vocabulary, embedding
    return pz.nn.Linear(pz.nn.Parameter(val, "input_embed"), ["vocabulary"], ["embedding"])
get_resids_one_hot = get_resids.select().at_instances_of(pz.nn.EmbeddingLookup).apply(rep_w_linear)
get_resids_one_hot_call = jit_wrapper.Jitted(get_resids_one_hot)

In [8]:
from micrlhf.sampling import sample, trange, jnp, load_tokenizer, jit_wrapper
from tqdm.auto import trange
from penzai.toolshed import sharding_util
import dataclasses
from functools import partial
import numpy as np
import jax

@jax.jit
def loss_fn(logits, inputs):
    losses = pz.nx.nmap(lambda l, i: jnp.take_along_axis(jax.nn.log_softmax(l[:-1], -1), i[1:, None], 1)[:, 0].mean()
                        )(logits.untag("seq", "vocabulary"), inputs.tokens.untag("seq"))
    return -losses

bs_start = llama.mesh.shape["dp"]
tokens_init = tokenizer.encode("<|user|>\nX X X X X X X X X X X X X X X X X X X X<|end|>\n<|assistant|>\n")
optim_mask = [token == 1060 for token in tokens_init]
tokens_init = np.asarray(tokens_init)
MAX_ELITES = 4
tokens_init = np.repeat(tokens_init[None, :], MAX_ELITES, axis=0)
seed = 23
np.random.seed(seed)
tokens_init[:, optim_mask] = np.random.randint(100, tokenizer.vocab_size, tokens_init[:, optim_mask].shape)
def tokens_to_array(tokens):
    token_array = jnp.asarray(tokens)
    if len(token_array) >= bs_start:
        token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    token_array = pz.nx.wrap(token_array, "batch", "seq")
    return token_array
def run_tokens(token_array, grad_metric=None):
    if not isinstance(token_array, pz.nx.NamedArray):
        token_array = tokens_to_array(token_array)
    inputs = llama.inputs.from_basic_segments(token_array)
    if grad_metric:
        @partial(jax.grad, has_aux=True)
        def lwg(x):
            logits, resids = get_resids_one_hot_call(dataclasses.replace(inputs, tokens=x))
            resids = {resid.tag: resid.value for resid in resids}
            metric = grad_metric(logits, resids, inputs)
            return metric, (logits, resids)
        vocab = llama.select().at_instances_of(pz.nn.EmbeddingLookup).get_sequence()[0].table.embeddings.value.named_shape["vocabulary"]
        one_hots = pz.nx.nmap(lambda x: jax.nn.one_hot(x, vocab))(inputs.tokens).tag("vocabulary")
        grad, (logits, resids) = lwg(one_hots)
    else:
        logits, resids = get_resids_call(inputs)
    losses = loss_fn(logits, inputs)
    if not grad_metric:
        resids = {resid.tag: resid.value for resid in resids}
    return_vals = logits, losses, resids
    if grad_metric:
        return_vals = return_vals + (grad,)
    return return_vals

mask = jax.device_put(jnp.asarray(optim_mask), jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("sp")))
# @partial(jax.jit, static_argnames=("key", "candidates", "expected_changes"))
def algo_iteration(elites, vector, key="resid_pre.16", candidates=128, seed=13, expected_changes=3, max_inv_temp=4, topk=128):
    elites = elites.untag("solutions").tag("batch")
    logits, _, _, grads = run_tokens(elites, grad_metric=lambda _l, r, _i: (r[key][{"seq": -1}].untag("embedding") * vector).sum().data_array.mean())
    grads = pz.nx.nmap(lambda x: x >= jax.lax.top_k(x, topk)[0][-1])(grads.untag("vocabulary")).tag("vocabulary")
    logits = logits.untag("batch").tag("elites")

    def temper(logits, key, elites, grads):
        key_choice, key_random = jax.random.split(key)
        index = jax.random.randint(key_choice, (), 0, len(logits) - 1)
        key_categorical, key_uniform, key_bernoulli, key_randint, key_use_grads, key_mutations = jax.random.split(key_random, 6)
        logit = logits[index]
        elite = elites[index]
        grads = grads[index]
        logit = jnp.roll(logit, 1, 0)
        logit = logit * jax.random.uniform(key_uniform, minval=0, maxval=max_inv_temp)
        use_grads = jax.random.bernoulli(key_use_grads, p=0.7).astype(jnp.int_)
        logit = jax.lax.switch(use_grads, ((lambda x: x), (lambda x: jnp.where(grads, x, -jnp.inf))), logit)
        to_change = jax.random.bernoulli(key_bernoulli, max(.5, expected_changes - 1) / mask.sum(), mask.shape)
        definite_indices = jax.random.randint(key_randint, mask.shape[:-1], 0, mask.shape[-1])
        definite_mask = jax.nn.one_hot(definite_indices, to_change.shape[-1], dtype=jnp.bool_)
        to_change = to_change | definite_mask
        return jnp.where(mask & to_change,
                         jax.random.categorical(key_categorical, logit),
                         elite)
    tempered_samples = pz.nx.nmap(temper)(
        logits.untag("elites", "seq", "vocabulary"),
        pz.nx.wrap(jax.random.split(jax.random.key(seed), candidates), "batch"),
        elites.untag("batch", "seq"),
        grads.untag("batch", "seq", "vocabulary")).tag("seq")
    tempered_samples = sharding_util.name_to_name_device_put(tempered_samples, llama.mesh, dict(batch="dp", seq="sp"))
    _, new_losses, new_resids = run_tokens(tempered_samples)
    new_scores = (new_resids[key][{"seq": -1}].untag("embedding") * vector).sum().astype(new_losses.dtype)
    metrics = pz.nx.nmap(lambda *xs: jnp.stack(xs))(new_losses, new_scores).tag("metrics")
    solution_axes = [k for k in tempered_samples.named_shape.keys() if k != "seq"]
    solutions = tempered_samples.untag(*solution_axes).flatten().tag("solutions").unwrap("solutions", "seq")
    metrics = metrics.untag(*(k for k in solution_axes if k != "seq")).flatten().tag("solutions").unwrap("solutions", "metrics")
    return solutions, metrics


best_metrics = None
best = tokens_to_array(tokens_init).untag("batch").tag("solutions")
xent_min = 0.1
xent_max = 1
weights = jnp.stack((
    jnp.linspace(-xent_max, -xent_min, MAX_ELITES),
    jnp.ones(MAX_ELITES),
), -1)
@partial(jax.jit, donate_argnums=(0, 1))
def combine_solutions(best_metrics, best, metrics, solutions):
    if best_metrics is not None:
        best_metrics = jnp.concatenate((best_metrics, metrics), 0)
        best = pz.nx.nmap(lambda a, b: jnp.concatenate((a, b)))(
            best.untag("solutions"),
            pz.nx.wrap(solutions, "solutions", "seq").untag("solutions")
        ).tag("solutions").unwrap("solutions", "seq")
    else:
        best_metrics = metrics
        best = solutions
    elite_mask = (best_metrics[None, :] * weights[:, None]).sum(-1).argmax(1)
    best_metrics = best_metrics[elite_mask]
    best = pz.nx.wrap(best[elite_mask], "solutions", "seq")
    return best_metrics, best
for seed in (bar := trange(1_000)):
    solutions, metrics = algo_iteration(best, vector, seed=seed)
    best_metrics, best = combine_solutions(best_metrics, best, metrics, solutions)
    m = {}
    for index in range(MAX_ELITES):
        i = index
        m |= {f"decoded.{i}": tokenizer.decode(best[{"solutions": index}].unwrap("seq")),
              f"loss.{i}": best_metrics[index][0], f"score.{i}": best_metrics[index][1]}
    bar.set_postfix(**m)

  0%|          | 0/1000 [00:00<?, ?it/s]

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/jax/_src/numpy/array_methods.py:68: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in astype is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return lax_numpy.astype(arr, dtype, copy=copy, device=device)


In [ ]:
_, resids = get_resids_call(inputs)